In [45]:
import numpy as np
import os, sys
import pickle, functools, operator
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import random #shuffle
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from netCDF4 import Dataset

## Data Processing

In [46]:
def CMIPdata(Xdata, Ydata, out , myform):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')
    sst_1 = np.zeros((myform,140,12,24,72))

    #首年序列
    #i= 21个模式
    for i in range(myform):
        sst_1[i,:,:,:,:] = inp1.variables['sst1'][1+141*i:141+141*i,0:12,:,:]
    #(21,140,12,24,72)
    #首年序列

    
    #flatted扁平化                                             
    sst_2 = np.zeros((myform,1680,24,72))
    for i in range(myform):
        for j in range(140):
            sst_2[i,j*12:(j+1)*12,:,:] = sst_1[i,j,:,:,:]
    
        #(21,1680,24,72)
        #每个模式 139*12+1 = 1680 个序列 序列长度为12

    #丢弃1个月 1862.1
    sst_2 = sst_2[:,1:,:,:]
    #(form,1679,24,72)


    winsize = 12
    #winnum = 1679-winsize +1 #1668
    """跳跃六个月滑窗 = 6"""
    ts=6 
    #(1679-12)/6+1 =27
    winnum=int((1679-12)/ts +1) 

    sst_3 = np.zeros((myform, winnum, winsize, 24, 72))#(form,1668, 3, 24, 72)
    

    sst_3 = np.zeros((myform,winnum,12,24,72))
    
    for i in range(myform):
        for j in range(winnum):
            #print(j)
            sst_3[i,j,:,:,:] = sst_2[i,(j*ts):(j*ts)+winsize,:,:]
            

    sst_4 = np.zeros((int(winnum*myform),12,24,72))
    #t300_4 = np.zeros((int(winnum*myform),12,24,72))
    for i in range(myform):
        sst_4[i*winnum:(i+1)*winnum,:,:,:] = sst_3[i,:,:,:,:]


    sst_4 = sst_4.swapaxes(1, 3)
    sst_4 = sst_4.swapaxes(1, 2)
    trX = sst_4
    del sst_1,sst_2,sst_3,sst_4
    #trX = trX.reshape(int(winnum*myform),12,1728)

    #保存np数组
    #(5859, 12, 24, 72, 2)
    #np.save("./CMIPdata/CMIP_trX_21_ts6_out.npy",trX) 

    #label
    #平铺
    pr_1 = np.zeros(((myform),1692))#21,1692
    for i in range(myform):
        for j in range(141):
            pr_1[i,j*12:(j+1)*12] = inp2.variables['pr'][(141*i)+j,:,0,0]
            
    #out
    pr_2 =  np.zeros((myform,winnum,out))#form,1668,1
    
    for i in range(myform):
        for j in range(winnum):
            pr_2[i,j,:] = pr_1[i,(j*ts):(j*ts)+out]

    trY = np.zeros((int(winnum*myform),out))#form*1668,1
    for i in range(myform):
        trY[i*winnum:(i+1)*winnum,:] = pr_2[i,:,:]

    #np.save("./CMIPdata/CMIP_trY_21_ts6_out%s.npy"%out,trY)
    trY_decoder_input = np.zeros((int(winnum*myform),out))
    trY_decoder_input[:,1:] = trY[:,:-1]

    #return trX[1:,:,:] ,trY.reshape(-1,24,1)[1:,:,:] ,trY_decoder_input
    return trX ,trY.reshape(-1,24,1) ,trY_decoder_input.reshape(-1,24,1)

In [9]:
def SODAdata(Xdata, Ydata, out):
    inp1 = Dataset(Xdata,'r')
    inp2 = Dataset(Ydata,'r')

    #time_step = 1 month                                               
    sst_1 = np.zeros((99,12,24,72))

    sst_1[:,:,:,:] = inp1.variables['sst'][1:,0:12,:,:]
    #(99,12,24,72)
    #1872-1970

    sst_2 = np.zeros((1188,24,72))

    for i in range(99):
        sst_2[i*12:(i+1)*12,:,:] = sst_1[i,:,:,:]
    #(1188,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_2 = sst_2[1:,:,:]

    sst_3 = np.zeros((1176,12,24,72))
    #滑窗
    for i in range(1176):
        sst_3[i:,:,:] = sst_2[i:i+12,:,:]
    #(1176,12,24,72)


    #channel = 2
    #trX = np.zeros((1176,12,24,72,2))
    #trX[:,:,:,:,0] = sst_3
    #trX[:,:,:,:,1] = t300_3
    sst_3 = sst_3.swapaxes(1, 3)
    sst_3 = sst_3.swapaxes(1, 2)
    trX = sst_3
    del sst_1,sst_2,sst_3
    #del t300_1,t300_2,t300_3

    #保存np数组
    #np.save("./SODAdata/SODA_trX_ts1_out.npy",trX)

    #label
    inpv2 = np.zeros((1200))
    for i in range(100):
        inpv2[i*12:(i+1)*12] = inp2.variables['pr'][i,:,0,0]
    #(1200)

    #out = 1
    trY = np.zeros((1176,out,1))
    #滑窗
    for i in range(1176):
        trY[i,:,0] = inpv2[i:i+out]   
    #(1176,out)
    trY_decoder_input = np.zeros((1176,24,1))
    trY_decoder_input[:,1:,:] = trY[:,:-1,:]

    #np.save("./SODAdata/SODA_trY_ts1_out%s.npy"%out,trY) 
    #(1176,12,24,72,2)
    #(1176,out,1)
    return trX ,trY, trY_decoder_input

In [47]:
def GOSDAdata(Xdata, Ydata, out):
    #test data
    inp11 = Dataset(Xdata,'r')
    inp22 = Dataset(Ydata,'r')
                                              
    sst_11 = np.zeros((33,12,24,72)) #1983-2015
    #t300_11 = np.zeros((33,12,24,72))

    sst_11[:,:,:,:] = inp11.variables['sst'][3:,0:12,:,:]
    #t300_11[:,:,:,:] = inp11.variables['t300'][3:,0:12,:,:]
    #(33,12,24,72)

    sst_22 = np.zeros((396,24,72))
    #t300_22 = np.zeros((396,24,72))

    for i in range(33):
        sst_22[i*12:(i+1)*12,:,:] = sst_11[i,:,:,:]
        #t300_22[i*12:(i+1)*12,:,:] = t300_11[i,:,:,:]
    #(396,24,72)
    #丢弃一个月，便于制作滑窗序列
    sst_22 = sst_22[1:,:,:]
    #t300_22 = t300_22[1:,:,:]#(395,24,72)

    #滑窗
    winsize = 12
    winnum = 395-winsize + 1 #384
    sst_33 = np.zeros((winnum, winsize, 24, 72))#(384, 12, 24, 72)
    #t300_33 = np.zeros((winnum, winsize, 24, 72))

    for i in range(winnum):
        sst_33[i,:,:,:] = sst_22[i:i+12,:,:]
        #t300_33[i,:,:,:] = t300_22[i:i+12,:,:]
    #(384,12,24,72)


    #channel = 2
    #testX = np.zeros((winnum,12,24,72,2))#384
    #testX[:,:,:,:,0] = sst_33
    #testX[:,:,:,:,1] = t300_33
    sst_33 = sst_33.swapaxes(1, 3)
    sst_33 = sst_33.swapaxes(1, 2)

    testX = sst_33#384
    #testX[:,:,:,1] = t300_33.reshape(384,12,1728)
    del sst_11,sst_22,sst_33
    #del t300_11,t300_22,t300_33
    #testX = testX.reshape(384,12,3456)

    #label
    inpv22 = np.zeros((408))#2017-1984 +1=34
    for i in range(34):
        inpv22[i*12:(i+1)*12] = inp22.variables['pr'][i+2,:,0,0]# +2:从1984开始
        #(408)

    testY = np.zeros((winnum,out,1))
    #out = 1
    #滑窗
    for i in range(winnum):
        testY[i,:,0] = inpv22[i:i+out]#24
    #(384,out,1)

    return testX, testY

In [11]:

trX, trY, trY_decoder_input= SODAdata('/home/d/Q/saconvlstm/SODA.input.36mon.1871_1970.nc', \
                                      '/home/d/Q/saconvlstm/SODA.label.12mon.1873_1972.nc', 24)

In [12]:

trX1, trY1, trY_decoder_input1 = CMIPdata('/home/d/Q/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

In [51]:
trX = np.append(trX, trX1, axis=0)
trY = np.append(trY, trY1, axis=0)
trY_decoder_input = np.append(trY_decoder_input, trY_decoder_input1, axis=0)

NameError: name 'trX1' is not defined

In [48]:
testX, testY = GOSDAdata('/home/d/Q/saconvlstm/GODAS.input.36mon.1980_2015.nc',\
                         '/home/d/Q/saconvlstm/GODAS.label.12mon.1982_2017.nc', 24)

In [52]:
indices = np.arange(trX.shape[0])
np.random.shuffle(indices)
trX = trX[indices]
trY = trY[indices]
trY_decoder_input = trY_decoder_input[indices]

In [7]:
#修改output长度
def set_output(Y, startout, endout):
    outlen = endout-startout+1
    trY = np.zeros((Y.shape[0],outlen,1))
    trY_decoder_input = np.zeros((Y.shape[0],outlen,1))
    
    trY = Y[:,(startout-1):endout,:]
    trY_decoder_input[:,1:,:] = trY[:,:-1,:]
    
    return trY, trY_decoder_input

In [7]:
trX.shape[0]

1176

In [50]:
trX, trY, trY_decoder_input = CMIPdata('/home/d/Q/saconvlstm/CMIP5.input.36mon.1861_2001.nc',\
                                          '/home/d/Q/saconvlstm/CMIP5.label.12mon.1863_2003.nc', 24,  21)

In [8]:
trY, trY_decoder_input = set_output(trY, 14, 17)

## Check data

In [53]:
print(testX.shape)
print(testY.shape)

(384, 24, 72, 12)
(384, 24, 1)


In [54]:
print(trX.shape)
print(trY.shape)
print(trY_decoder_input.shape)

(5838, 24, 72, 12)
(5838, 24, 1)
(5838, 24, 1)


In [12]:
trY[2000,:,:]

array([[ 0.1671124 ],
       [ 0.1033603 ],
       [-0.04715098],
       [-0.34032083],
       [-0.65523213],
       [-0.88238829],
       [-1.03931367],
       [-1.25271392],
       [-1.56010616],
       [-1.87568974],
       [-1.98356259],
       [-1.91057312],
       [-1.7098825 ],
       [-1.49061811],
       [-1.26539171],
       [-1.0021739 ],
       [-0.67665362],
       [-0.3104468 ],
       [-0.03180949],
       [ 0.24412023],
       [ 0.46249807],
       [ 0.69632208],
       [ 0.86289573],
       [ 0.92010337]])

In [13]:
trY_decoder_input[2000,:,:]

array([[ 0.        ],
       [ 0.1671124 ],
       [ 0.1033603 ],
       [-0.04715098],
       [-0.34032083],
       [-0.65523213],
       [-0.88238829],
       [-1.03931367],
       [-1.25271392],
       [-1.56010616],
       [-1.87568974],
       [-1.98356259],
       [-1.91057312],
       [-1.7098825 ],
       [-1.49061811],
       [-1.26539171],
       [-1.0021739 ],
       [-0.67665362],
       [-0.3104468 ],
       [-0.03180949],
       [ 0.24412023],
       [ 0.46249807],
       [ 0.69632208],
       [ 0.86289573]])

## Model

In [55]:

w4 = [30 * 18 * 6, 30]

input_cnn = Input(shape=(24, 72,1))

x = layers.Conv2D(30, (8,4), activation='tanh', padding='SAME')(input_cnn)
x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = layers.Conv2D(30, (4,2), activation='tanh', padding='SAME')(x)
x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

x = layers.Conv2D(30, (4,2), activation='tanh', padding='SAME')(x)
x = tf.reshape(x, [-1, w4[0]])
outputs_cnn = x
model_cnn = Model(input_cnn,outputs_cnn)
model_cnn.summary()
#tf.keras.utils.plot_model(model_cnn, to_file='model_train.png', show_shapes=True, show_layer_names=True)

Model: "functional_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        [(None, 24, 72, 1)]       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 72, 30)        990       
_________________________________________________________________
tf_op_layer_MaxPool_4 (Tenso [(None, 12, 36, 30)]      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 36, 30)        7230      
_________________________________________________________________
tf_op_layer_MaxPool_5 (Tenso [(None, 6, 18, 30)]       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 18, 30)         7230      
_________________________________________________________________
tf_op_layer_Reshape_40 (Tens [(None, 3240)]          

In [56]:
_input = Input(shape=(24, 72, 12))
x1 = model_cnn(_input[:,:,:,0])
x2 = model_cnn(_input[:,:,:,1])
x3 = model_cnn(_input[:,:,:,2])
x4 = model_cnn(_input[:,:,:,3])
x5 = model_cnn(_input[:,:,:,4])
x6 = model_cnn(_input[:,:,:,5])
x7 = model_cnn(_input[:,:,:,6])
x8 = model_cnn(_input[:,:,:,7])
x9 = model_cnn(_input[:,:,:,8])
x10 = model_cnn(_input[:,:,:,9])
x11 = model_cnn(_input[:,:,:,10])
x12 = model_cnn(_input[:,:,:,11])

x1 = tf.reshape(x1, [-1, 1, 3240])
x2 = tf.reshape(x2, [-1, 1, 3240])
x3 = tf.reshape(x3, [-1, 1, 3240])
x4 = tf.reshape(x4, [-1, 1, 3240])
x5 = tf.reshape(x5, [-1, 1, 3240])
x6 = tf.reshape(x6, [-1, 1, 3240])
x7 = tf.reshape(x7, [-1, 1, 3240])
x8 = tf.reshape(x8, [-1, 1, 3240])
x9 = tf.reshape(x9, [-1, 1, 3240])
x10 = tf.reshape(x10, [-1, 1, 3240])
x11 = tf.reshape(x11, [-1, 1, 3240])
x12 = tf.reshape(x12, [-1, 1, 3240])

output = layers.concatenate([x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12], axis=1)#(None, 12,3240)

In [57]:
time_steps_encoder=4
#num_encoder_tokens=3456
num_encoder_tokens=3240
latent_dim=512
time_steps_decoder=24
#time_steps_decoder=3
num_decoder_tokens=1

In [58]:
# Setting up the encoder
encoder_inputs = output
"""后为特征数，前为批次数量！！！"""
#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output1, state_h1, state_c1 = encoder_lstm1(encoder_inputs)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)
# 我们抛弃掉`encoder_outputs`因为我们只需要LSTM cell的内部state参数
encoder_states = [state_h, state_c]

# Set up the decoder
decoder_inputs = Input(shape=(time_steps_decoder, num_decoder_tokens), name= "decoder_inputs")
# 我们设定我们的解码器回传整个输出的序列同时也回传内部的states参数
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name='decoder_lstm')
# 我们使用`encoder_states`来做为初始值(initial state) <-- 重要
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense1 = Dense(latent_dim, activation ='tanh')
decoder_outputs = decoder_dense1(decoder_outputs)
decoder_dense = Dense(num_decoder_tokens, name='decoder_out')
decoder_outputs = decoder_dense(decoder_outputs)

# 定义一个模型接收encoder_input_data` & `decoder_input_data`做为输入而输出`decoder_target_data`
model = Model([_input, decoder_inputs], decoder_outputs)
#model.summary()
#plot_model(model, to_file='model_train.png', show_shapes=True, show_layer_names=True)

## Train

In [73]:
def custome_rmse_fn(y_true, y_pred):
    """ custome loss function
    The 24 series is not equally weighted, so log1p weighted is used.
    This is just my initial try, still have further improvement space.

    y_true: batch * 24
    """ 
    #y_pred = y_pred.numpy().reshape(-1,24)
    #y_true = y_true.numpy().reshape(-1,24)
    diff = (y_pred - y_true) ** 2
    predict_sequence_length = tf.shape(y_true)[-1]#获取序列长度
    alpha = [np.log1p(i) for i in range(1, time_steps_decoder+1)]
    #alpha = [np.log(i)*j for i,j in zip(range(1, predict_sequence_length+1), [0.65]*4+[1]*7+[1.2]*7+[1.5]*6)]
    alpha = tf.reshape(tf.convert_to_tensor(alpha, tf.float32), (1, time_steps_decoder))
    #reshape(1,24)
    rmse = tf.sqrt(tf.reduce_mean(diff * alpha))
    return rmse

In [75]:
epochs=100
# Early Stopping
earlystopping = EarlyStopping(monitor='val_loss', patience = 5, restore_best_weights = True)

# Tensorboard callback
#logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
#tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

# Run training
#opt = keras.optimizers.Adam(lr = 0.0001)
#RMSprop
#当学习停滞时，减少2倍或10倍的学习率常常能获得较好的效果。
#x = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,patience=2,verbose=0,mode="auto")

model.compile(metrics=['mae'], optimizer='Adam', loss='mse')

model.fit([trX, trY_decoder_input], trY, batch_size=64, epochs=epochs, validation_split=0.3, verbose=1, callbacks=[earlystopping, ])

Epoch 1/100
64/64 [==============================] - 46s 723ms/step - loss: 0.1567 - mae: 0.0764 - val_loss: 0.1339 - val_mae: 0.0634
Epoch 2/100
64/64 [==============================] - 45s 709ms/step - loss: 0.1229 - mae: 0.0598 - val_loss: 0.1315 - val_mae: 0.0623
Epoch 3/100
64/64 [==============================] - 46s 712ms/step - loss: 0.1200 - mae: 0.0588 - val_loss: 0.1288 - val_mae: 0.0615
Epoch 4/100
64/64 [==============================] - 46s 712ms/step - loss: 0.1187 - mae: 0.0582 - val_loss: 0.1299 - val_mae: 0.0618
Epoch 5/100
64/64 [==============================] - 46s 711ms/step - loss: 0.1161 - mae: 0.0571 - val_loss: 0.1300 - val_mae: 0.0619
Epoch 6/100
64/64 [==============================] - 46s 712ms/step - loss: 0.1152 - mae: 0.0567 - val_loss: 0.1300 - val_mae: 0.0619
Epoch 7/100
64/64 [==============================] - 45s 710ms/step - loss: 0.1145 - mae: 0.0564 - val_loss: 0.1290 - val_mae: 0.0614
Epoch 8/100
64/64 [==============================] - 46s 711ms

## Save Model

In [76]:
save_model_path = 'model_final'
if not os.path.exists(save_model_path):
    os.makedirs(save_model_path)

# Saving encoder as in training
encoder_model = Model(_input, encoder_states)

# Saving decoder states and dense layer 
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense1(decoder_outputs)
decoder_outputs = decoder_dense(decoder_outputs)
#modle(input, output)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

encoder_model.summary()
decoder_model.summary()
encoder_model.save(os.path.join(save_model_path, 'encoder_model.h5'))
decoder_model.save_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

Model: "functional_38"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           [(None, 24, 72, 12)] 0                                            
__________________________________________________________________________________________________
tf_op_layer_strided_slice_36 (T [(None, 24, 72)]     0           input_21[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_strided_slice_37 (T [(None, 24, 72)]     0           input_21[0][0]                   
__________________________________________________________________________________________________
tf_op_layer_strided_slice_38 (T [(None, 24, 72)]     0           input_21[0][0]                   
______________________________________________________________________________________

## Load Model

In [77]:
# inference encoder model
save_model_path = 'model_final'
inf_encoder_model = load_model(os.path.join(save_model_path, 'encoder_model.h5'))

# inference decoder model
decoder_inputs = Input(shape=(None, num_decoder_tokens))#注意是none (none, 1)
decoder_dense1 = Dense(latent_dim,activation ='tanh')
decoder_dense = Dense(num_decoder_tokens)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs11 = decoder_dense1(decoder_outputs)
decoder_outputs = decoder_dense(decoder_outputs11)

inf_decoder_model = Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
inf_decoder_model.load_weights(os.path.join(save_model_path, 'decoder_model_weights.h5'))

def decode_predict(input_seq):

    # 将输入编码为状态向量，states_value用于decoder
    states_value = inf_encoder_model.predict(input_seq.reshape(-1, 24, 72, 12))
    target_seq = np.zeros((1, 1, num_decoder_tokens))#(1, 1, 1)
    target_seq[0, 0, 0] = 0.
    decoded_sentence = []
    
    for i in range(24):
      #print(target_seq)
      output, h, c = inf_decoder_model.predict([target_seq] + states_value)
      #加入预测值
      decoded_sentence.append(output[0,0,0])
      
      
      # 更新目标序列（长度为1）。
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq = output
      
      # Update states
      states_value = [h, c]

    return decoded_sentence

## Project


In [78]:
out = []
for i in range(testX.shape[0]):
    print("test No." + str(i+1))
    out.append(decode_predict(testX[i,:,:]))

test No.1
test No.2
test No.3
test No.4
test No.5
test No.6
test No.7
test No.8
test No.9
test No.10
test No.11
test No.12
test No.13
test No.14
test No.15
test No.16
test No.17
test No.18
test No.19
test No.20
test No.21
test No.22
test No.23
test No.24
test No.25
test No.26
test No.27
test No.28
test No.29
test No.30
test No.31
test No.32
test No.33
test No.34
test No.35
test No.36
test No.37
test No.38
test No.39
test No.40
test No.41
test No.42
test No.43
test No.44
test No.45
test No.46
test No.47
test No.48
test No.49
test No.50
test No.51
test No.52
test No.53
test No.54
test No.55
test No.56
test No.57
test No.58
test No.59
test No.60
test No.61
test No.62
test No.63
test No.64
test No.65
test No.66
test No.67
test No.68
test No.69
test No.70
test No.71
test No.72
test No.73
test No.74
test No.75
test No.76
test No.77
test No.78
test No.79
test No.80
test No.81
test No.82
test No.83
test No.84
test No.85
test No.86
test No.87
test No.88
test No.89
test No.90
test No.91
test No.

In [ ]:
testY.shape

In [80]:
cor = np.zeros((24))
for i in range(24):
    cor[i] = np.corrcoef(testY.reshape(384,24)[:,i],np.array(out)[:,i])[0,1]
print(cor)

[0.97301957 0.93920321 0.89352694 0.83643479 0.76734964 0.68809902
 0.6064623  0.529289   0.46235577 0.41001724 0.37888119 0.37051364
 0.37991741 0.39346962 0.39440348 0.37316187 0.33479706 0.2910739
 0.25256526 0.22493691 0.21004243 0.20587179 0.21190604 0.22543228]


In [81]:
testY[3]

array([[-0.86499506],
       [-0.88665622],
       [-0.82672739],
       [-0.62346232],
       [-0.70335156],
       [-0.9443807 ],
       [-1.13839138],
       [-1.15875304],
       [-1.15833545],
       [-1.18053198],
       [-1.17541635],
       [-1.00110507],
       [-0.80188906],
       [-0.6772756 ],
       [-0.52563614],
       [-0.43219197],
       [-0.36762857],
       [-0.33536553],
       [-0.28663447],
       [-0.24307424],
       [-0.35159171],
       [-0.53687763],
       [-0.68450022],
       [-0.63430518]])

In [71]:
out[3]

[-0.7871893,
 -0.7362599,
 -0.65568805,
 -0.577988,
 -0.5031866,
 -0.428288,
 -0.3543526,
 -0.28536102,
 -0.22521791,
 -0.17614868,
 -0.13867389,
 -0.112198554,
 -0.09551874,
 -0.0870989,
 -0.08523147,
 -0.08817759,
 -0.09430601,
 -0.10220613,
 -0.11075708,
 -0.119151965,
 -0.12688662,
 -0.13372481,
 -0.13964869,
 -0.14480345]

In [25]:
with open("/home/d/Q/result.txt","a") as f:
    f.write(str(cor) + "\n")

## Correlation coefficient